<a href="https://colab.research.google.com/github/trantoan62/AI_FinalReport/blob/main/Predict_fromvideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall h5py -y
!pip install h5py==2.10.0
!pip uninstall numpy -y
!pip install numpy==1.16.4
!pip uninstall scikit-image -y
!pip install scikit-image==0.16.2
!pip install --no-deps tensorflow==1.15.3
!pip install --no-deps keras==2.2.4

In [ ]:
 !git clone https://github.com/matterport/Mask_RCNN.git
 %cd Mask_RCNN
!pip install -r requirements.txt
!python setup.py install
%cd ..

In [20]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

In [21]:
import cv2
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import glob
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
class PredictionConfig(Config):
	NAME = "cow_cfg"
	NUM_CLASSES = 1 + 1
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

In [24]:
cfg = PredictionConfig()
%cd /content/drive/MyDrive
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
model.load_weights('mask_rcnn_cow_cfg_0003.h5', by_name=True)

/content/drive/MyDrive


In [6]:
def print_fps(video):
  (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
  if int(major_ver)  < 3 :
      fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
      print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
  else :
      fps = video.get(cv2.CAP_PROP_FPS)
      print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

def make_video(outvid, images=None, fps=30, size=None, is_color=True, format="FMP4"):
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

In [ ]:
%cd /content/drive/MyDrive
capture = cv2.VideoCapture("video3.mp4")
print_fps(capture)
try:
    if not os.path.exists("./results"):
        os.makedirs("./results")
except OSError:
    print ('Error: Creating directory of data')
frames = []
frame_count = 0

while True:
    ret, frame = capture.read()
    if not ret:
        break
    frame_count += 1
    frames.append(frame)
    print('frame_count :{0}'.format(frame_count))
    if len(frames) == 1:
        results = model.detect(frames, verbose=0)
        print('Predicted')
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            cow = 0
            for box in r['rois']:
              cow += 1
              y1, x1, y2, x2 = box
              cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
              cv2.putText(frame,'Cow #' + str(cow), (x1 + 2, y1 + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2, 2)
            cv2.putText(frame,'Num of cow:' + str(cow), (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, 2)
            name = '{0}.jpg'.format(frame_count + i)
            name = os.path.join("./results", name)
            cv2.imwrite(name, frame)
        # Clear the frames array to start the next batch
        frames = []

capture.release()

/content/drive/MyDrive
Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.0
frame_count :1
Predicted
frame_count :2
Predicted
frame_count :3
Predicted
frame_count :4
Predicted
frame_count :5
Predicted
frame_count :6
Predicted
frame_count :7
Predicted
frame_count :8
Predicted
frame_count :9
Predicted
frame_count :10
Predicted
frame_count :11
Predicted
frame_count :12
Predicted
frame_count :13
Predicted
frame_count :14
Predicted
frame_count :15
Predicted
frame_count :16
Predicted
frame_count :17
Predicted
frame_count :18
Predicted
frame_count :19
Predicted
frame_count :20
Predicted
frame_count :21
Predicted
frame_count :22
Predicted
frame_count :23
Predicted
frame_count :24
Predicted
frame_count :25
Predicted
frame_count :26
Predicted
frame_count :27
Predicted
frame_count :28
Predicted
frame_count :29
Predicted
frame_count :30
Predicted
frame_count :31
Predicted
frame_count :32
Predicted
frame_count :33
Predicted
frame_count :34
Predicted
frame_count :35
Predicted
frame_count :36
P

In [9]:
import glob
import os
%cd /content/drive/MyDrive
images = list(glob.iglob(os.path.join("./results", '*.*')))
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
outvid = os.path.join("./video", "video_results1.mp4")
make_video(outvid, images, fps=20)

/content/drive/MyDrive


<VideoWriter 0x7fac94837230>